# Late Fusion Concatenation Model
Inspired by Homework 2

In [4]:

# we want to load in a pretrained resnet model.
# we want to use the ImageFolder format specified by PyTorch
# we freeze the resnet parameters and train on our new dataset.
# train and evaluate
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import json
import glob
import itertools
from PIL import Image
from PIL.Image import BILINEAR
from torchinfo import summary
from transformers import (
    AutoImageProcessor,
    TrainingArguments,
    Trainer,
    ResNetForImageClassification,
    Owlv2VisionModel,
    ResNetModel,
    AutoProcessor
)
import evaluate
import accelerate


cudnn.benchmark = True
plt.ion()   # interactive mode

## Setting Up Dataset

In [5]:
# For straightforward datasets, sometimes you can make do with built-in PyTorch dataset objects.
# We want to apply automated data augmentations, which will be different for the training
# and eval scenarios

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
    ]),
}

In [6]:
data_dir = "../../data/dataset/"
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}

class_names = image_datasets['train'].classes

print(class_names)

print(image_datasets.keys())
print("label", image_datasets['train'][0][1])
# we will use this test image to do all our preliminary testing to make sure stuff works.
test_image = image_datasets['train'][0][0]
test_image = test_image.unsqueeze(0)
test_image.mean(), test_image.std(), test_image.shape

['1', '2', '3', '4', '5']
dict_keys(['train', 'val'])
label 0


(tensor(0.3929), tensor(0.2438), torch.Size([1, 3, 224, 224]))

## Baseline Concatenation Model

In [7]:

## OK
## We need to just make this take in the pooled output of a OwLViT and a YOLO model, along with a resnet.
## Early fusion model.
#device = "cpu"  
class BaseLineModel(nn.Module):
  def __init__(self,
               vit,
               resnet,
               tokenizer,
               device):
    super().__init__()

    self.vit = vit
    self.vit.eval()
    self.vit.to(device)
    
    self.resnet = resnet
    self.resnet.to(device)
    self.resnet.eval()

    self.tokenizer = tokenizer
    self.device = device
    
    self.concatenatedLayerSize = vit.config.hidden_size + 1000
    self.clf = nn.Linear(self.concatenatedLayerSize, 5)
    print(self.concatenatedLayerSize)

  def forward(self, images):
      # Computing image embeddings
      images = images.to(self.device)
      image_embeddings = self.resnet(images).logits
      print("image embeddings shape: ", image_embeddings.shape)
      
      # Computing caption embeddings
      # tokenize all captions
      inputs = self.tokenizer(images = images, return_tensors="pt", do_rescale=False).to(self.device)
      #Pass the tokenized captions through the BERT model
      vit_output = self.vit(**inputs)

      #get the pooler output from the BERT model's output
      pooled_output = vit_output.pooler_output

      # Concatenate image and caption embeddings along the batch dimension
      full_embeddings = torch.cat((image_embeddings, pooled_output), dim=1)

      print(full_embeddings.shape)
      return self.clf(full_embeddings)
   

## Load in pretrained Resnet and OwLViT Models

In [8]:
resnet = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")
vit = Owlv2VisionModel.from_pretrained("google/owlv2-base-patch16")
processor = AutoProcessor.from_pretrained("google/owlv2-base-patch16")


with torch.no_grad():
    output = resnet(test_image)
    print(output.logits.shape)




torch.Size([1, 1000])


In [9]:
print(test_image.shape)

torch.Size([1, 3, 224, 224])


In [10]:
concatModel = BaseLineModel(vit = vit, resnet = resnet, tokenizer = processor, device = "cpu")

concatModel(test_image)

1768
image embeddings shape:  torch.Size([1, 1000])
torch.Size([1, 1768])


tensor([[ 1.6318,  2.4691, -3.3295, -0.3652,  3.9658]],
       grad_fn=<AddmmBackward0>)

In [7]:
inputs = processor(images=test_image, return_tensors="pt", do_rescale = False)
outputs = vit(**inputs)

last_hidden_state = outputs.last_hidden_state
pooled_output = outputs.pooler_output  # pooled CLS states

AttributeError: 'Owlv2VisionModel' object has no attribute 'hidden_size'

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

